## Steps to run the model

1. Run the first cell directly below this one to install all needed packages and login your drive.
2. Restart the runtime and run the notebook from `Runtime -> Restart session and run all` (important!)
3. Run the buttons one by one
4. In case of any errors, start the session afresh by running (2)


In [2]:
# @title
from google.colab import drive
drive.mount('/content/drive')       #mount drive to enable fetching artifacts
!pip install scikit-learn==1.3.2 -q #match scikit learn version used in pickling model

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 103.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 89.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


# Environment Setup

In [3]:
# @title
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import pickle
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

# Create a stylish button with an icon and custom color
init_button = widgets.Button(
    description='Init Workspace',
    button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
    layout=widgets.Layout(width='150px', height='50px'),
    style={'font_weight': 'bold', 'font_size': '16px'}
)

# Output area to show result
output = widgets.Output()

# Function to handle button click
def on_init_click(b):
    with output:
        clear_output()
        #display(HTML('<h3 style="color:limegreen;"></h3>'))
        #import all needed packages.
        display(HTML('<h3 style="color:limegreen;">✅ Workspace Initialized</h3>'))


init_button.on_click(on_init_click)

# Display everything
display(init_button, output)


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [4]:
!pip uninstall numpy tensorflow keras -y


Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: tensorflow 2.18.0
Uninstalling tensorflow-2.18.0:
  Successfully uninstalled tensorflow-2.18.0
Found existing installation: keras 3.8.0
Uninstalling keras-3.8.0:
  Successfully uninstalled keras-3.8.0


In [5]:
!pip install numpy==1.21.6

ERROR: Ignored the following versions that require a different python version: 1.21.2 Requires-Python >=3.7,<3.11; 1.21.3 Requires-Python >=3.7,<3.11; 1.21.4 Requires-Python >=3.7,<3.11; 1.21.5 Requires-Python >=3.7,<3.11; 1.21.6 Requires-Python >=3.7,<3.11
ERROR: Could not find a version that satisfies the requirement numpy==1.21.6 (from versions: 1.3.0, 1.4.1, 1.5.0, 1.5.1, 1.6.0, 1.6.1, 1.6.2, 1.7.0, 1.7.1, 1.7.2, 1.8.0, 1.8.1, 1.8.2, 1.9.0, 1.9.1, 1.9.2, 1.9.3, 1.10.0.post2, 1.10.1, 1.10.2, 1.10.4, 1.11.0, 1.11.1, 1.11.2, 1.11.3, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 1.13.3, 1.14.0, 1.14.1, 1.14.2, 1.14.3, 1.14.4, 1.14.5, 1.14.6, 1.15.0, 1.15.1, 1.15.2, 1.15.3, 1.15.4, 1.16.0, 1.16.1, 1.16.2, 1.16.3, 1.16.4, 1.16.5, 1.16.6, 1.17.0, 1.17.1, 1.17.2, 1.17.3, 1.17.4, 1.17.5, 1.18.0, 1.18.1, 1.18.2, 1.18.3, 1.18.4, 1.18.5, 1.19.0, 1.19.1, 1.19.2, 1.19.3, 1.19.4, 1.19.5, 1.20.0, 1.20.1, 1.20.2, 1.20.3, 1.21.0, 1.21.1, 1.22.0, 1.22.1, 1.22.2, 1.22.3, 1.22.4, 1.23.0, 1.23.1, 1.23.2, 1.23.3, 1.23

In [ ]:
# @title
from tensorflow import keras
import pickle
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import os

# Set base directory
base_dir = '/content/drive/MyDrive/11-thesis-projects/engr-sgml-phd/artifacts'

# Buttons and output display
load_button = widgets.Button(
    description='Load Model',
    button_style='info',
    layout=widgets.Layout(width='150px', height='50px'),
    style={'font_weight': 'bold', 'font_size': '16px'}
)
load_output = widgets.Output()

model = None
scaler = None
cnn_fex = None
test_set = None #(X_test, y_test)

# Function to load the model
def on_load_model_click(b):
    global cnn_fex, scaler, model, test_set
    with load_output:
        clear_output()
        test_set_filename = os.path.join(base_dir, 'test_set.pkl')
        scaler_filename = os.path.join(base_dir, 'minmax_scaler.pkl')
        model_filename = os.path.join(base_dir, 'rf_classifier.pkl')
        feature_extractor_filename = os.path.join(base_dir, 'cnn_feature_extractor.keras')

        try:
            cnn_fex = keras.models.load_model(feature_extractor_filename)
            with open(model_filename, 'rb') as file:
                model = pickle.load(file)
            with open(scaler_filename, 'rb') as file:
                scaler = pickle.load(file)
            with open(test_set_filename, "rb") as f:
                test_set = pickle.load(f)
            display(HTML('<h3 style="color:limegreen;">✅ Artifacts loaded in workspace:</h3>'))
            display(HTML(f'<p><b>📦 Scaler:</b> {scaler_filename}</p>'))
            display(HTML(f'<p><b>📦 Classifier:</b> {model_filename}</p>'))
            display(HTML(f'<p><b>📦 Test Set:</b> {test_set_filename}</p>'))
            display(HTML(f'<p><b>📦 Feature Extractor:</b> {feature_extractor_filename}</p>'))
        except Exception as e:
            display(HTML(f'<h3 style="color:red;">❌ Failed to load model: {str(e)}</h3>'))

load_button.on_click(on_load_model_click)
display(load_button, load_output)


Button(button_style='info', description='Load Model', layout=Layout(height='50px', width='150px'), style=Butto…

Output()

# Generation of Meter Readings

In [ ]:
# @title

import pickle
import numpy as np
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
# ===============================
# Section 1: Mode + Alpha control
# ===============================

# Radio buttons for mode selection
mode_selector = widgets.RadioButtons(
    options=['determined', 'random'],
    description='Select Mode:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='200px')
)

# Alpha slider with uniform style
alpha_slider = widgets.FloatSlider(
    value=0.5,
    min=0.0,
    max=1.0,
    step=0.01,
    description='Alpha:',
    continuous_update=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

# Container to hold mode settings
mode_settings_box = widgets.VBox(
    [widgets.HTML("<h3>Mode Settings</h3>"), mode_selector],
    layout=widgets.Layout(padding='10px', width='100%')
)

# Output area to show/hide alpha slider and sliders
mode_output = widgets.Output()

# ===============================
# Section 2: Transformation sliders
# ===============================

def h(x, proportions):
    np.random.seed(42)
    #squeeze 3d inputs
    if x.ndim == 3 and x.shape[-1] == 1:
        x = x.squeeze(-1)
    anomalous_x = np.zeros_like(x)

    for i in range(x.shape[0]):
        method = np.random.choice(np.arange(1, 7), p=proportions)
        daily_data = x[i].copy()

        if method == 1:
            alpha = np.random.uniform(0.1, 0.8)
            anomalous_x[i] = alpha * daily_data
        elif method == 2:
            min_off = 4
            start = np.random.randint(0, 24 - min_off)
            duration = np.random.randint(min_off, 24 - start + 1)
            end = start + duration
            daily_data[start:end] = 0
            anomalous_x[i] = daily_data
        elif method == 3:
            gammas = np.random.uniform(0.1, 0.8, size=24)
            anomalous_x[i] = gammas * daily_data
        elif method == 4:
            daily_mean = np.mean(daily_data)
            gammas = np.random.uniform(0.1, 0.8, size=24)
            anomalous_x[i] = gammas * daily_mean
        elif method == 5:
            daily_mean = np.mean(daily_data)
            anomalous_x[i] = np.full(24, daily_mean)
        elif method == 6:
            anomalous_x[i] = daily_data[::-1]
    return anomalous_x[..., np.newaxis]

# Sliders for method proportions
method_sliders = [
    widgets.FloatSlider(
        value=0.17, min=0, max=1, step=0.01, description=f'Method {i}',
        continuous_update=False,
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='50%')
    ) for i in range(1, 7)
]

def get_normalized_proportions():
    total = sum(slider.value for slider in method_sliders)
    if total == 0:
        total = 1
    return [slider.value / total for slider in method_sliders]

# Output plot area
generate_output = widgets.Output()

def plot_sample(sample):
    plt.figure(figsize=(8, 6))
    plt.imshow(sample, cmap='gray', aspect='auto')
    plt.colorbar()
    plt.title("Sample Energy Usage Profile")
    plt.xlabel("Hour")
    plt.ylabel("Sample Index")
    plt.show()

example_sample = (None, None)

def generate_determined_sample():
    global example_sample
    with generate_output:
        clear_output(wait=True)
        X_test, y_test = test_set
        index = np.random.randint(0, len(X_test))
        example_sample = (X_test[index], int(y_test[index]))
        plot_sample(example_sample[0])

def generate_random_sample():
    global example_sample
    with generate_output:
        clear_output(wait=True)
        proportions = get_normalized_proportions()
        output_like = test_set[0][0]

        # The label is always None because it is manually
        # constructed and we cannot say what category it falls in
        example_sample = (h(output_like, proportions), None)
        plot_sample(example_sample[0])

def on_generate_click(button):
    if mode_selector.value == 'determined':
        generate_determined_sample()
    else:  # random mode
        generate_random_sample()

def on_mode_change(change):
    with mode_output:
        clear_output(wait=True)
        children = [widgets.HTML("<h3>Mode Settings</h3>"), mode_selector]

        if change['new'] == 'random':
            #controls sliders widht
            for slider in method_sliders:
                slider.layout.width = '400px'  # Set explicit width

            children.extend([
                widgets.HTML("<h3>Set the Proportions for Each Generation Method</h3>"),
                *method_sliders
            ])

        display(widgets.VBox(children, layout=widgets.Layout(width='100%')))

mode_selector.observe(on_mode_change, names='value')

# Generate button
generate_button = widgets.Button(
    description='Generate',
    button_style='success',
    layout=widgets.Layout(width='150px', height='50px'),
    style={'font_weight': 'bold', 'font_size': '16px'}
)
generate_button.on_click(on_generate_click)

# Initial display of mode section
with mode_output:
    display(mode_settings_box)

# Final layout
ui_elements = widgets.VBox(
    [
        mode_output,
        generate_button
    ],
    layout=widgets.Layout(
        align_items='flex-start',
        width='100%',
        padding='10px'
    )
)

display(ui_elements, generate_output)

Output()

# Classification of Energy Usage Profiles To Detect Theft
###*Note: Anamolous consumption patterns are labelled 1 while normal patterns are labelled 0.

In [ ]:
# @title
inference_button = widgets.Button(
    description='Run Inference',
    button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
    layout=widgets.Layout(width='150px', height='50px'),
    style={'font_weight': 'bold', 'font_size': '16px'}
)

inference_output = widgets.Output()

def on_inference_click(b):
    with inference_output:
        clear_output()
        #apply the scaler
        sample_reshaped = example_sample[0].reshape(-1, 24)
        sample_scaled = scaler.transform(sample_reshaped).reshape(example_sample[0].shape)
        sample_input = sample_scaled[np.newaxis, ..., np.newaxis]  # Shape: (1, n_days, 24, 1)

        #spply the feature extractor
        features = cnn_fex.predict(sample_input)

        #apply the classification model
        result = model.predict(features)
        predicted_label = int(result[0] > 0.5)  # Convert to 0 or 1

        if predicted_label == 1:
            display(HTML('<h3 style="color:red;">⚠️ Usage profile flagged <b>anomalous</b>.</h3>'))
        else:
            display(HTML('<h3 style="color:green;">✅ Usage profile flagged <b>normal</b>.</h3>'))

        if example_sample[1] == None:
          display(HTML('<hr><h4 style="color:gray;">Real label = <b>None<b></h4>'))
        elif example_sample[1] > 0:
          display(HTML('<hr><h4 style="color:gray;">Real label = <b>anomalous</b></h4>'))
        else:
          display(HTML('<hr><h4 style="color:gray;">Real label = <b>normal</b></h4>'))



inference_button.on_click(on_inference_click)

display(inference_button, inference_output)


Button(button_style='success', description='Run Inference', layout=Layout(height='50px', width='150px'), style…

Output()